This is a random simulation and does not always return the same result. Therefore this is just for show do not use this as a accurate estimator

In [11]:
import random
def winning_simulation(target = 100, p_win=0.6):
    rounds = 0
    winnings = 0 
    while winnings < target:
        rounds += 1
        if random.random() < p_win:
            winnings += 1
        else:
            winnings -= 1
    return rounds

rounds = winning_simulation()
print('Rounds needed to win 100 dollars:', rounds)

Rounds needed to win 100 dollars: 434


Using Chebyshev's Inequality or Hoefding's Inequality. Confidence was not specified in the original problem so the output is givene using 3 different confidence levels

In [ ]:
import math

# Parameters defined by problem 
# Note P(X=+1) = 0.6,  P(X=-1) = 1-0.6 = 0.4
T = 100           # target winnings
p = 0.6           # P(+1)
mu = 2*p - 1      # E[X] = 0.2
sigma2 = 1 - mu**2  # Var(X) = 0.96
n_min = math.floor(T/mu) + 1 

def chebyshev_tail(n, mu=mu, sigma2=sigma2, T=T):
    if n <= T/mu:
        return 1.0
    t = n*mu - T
    return (n*sigma2) / (t*t)

def hoeffding_tail(n, mu=mu, T=T):
    if n <= T/mu:
        return 1.0
    eps = mu - T/n
    return math.exp(-2 * n * eps*eps)

def n_hoeffding_for_target(delta, T=T, p=p):
    mu = 2*p - 1
    assert mu > 0
    # Exponential growth to find an upper bracket, then binary search
    lo = math.floor(T/mu) + 1
    n = lo
    while hoeffding_tail(n, mu=mu, T=T) > delta:
        n *= 2
    hi = n
    # Binary search for smallest n
    while lo < hi:
        mid = (lo + hi) // 2
        if hoeffding_tail(mid, mu=mu, T=T) <= delta:
            hi = mid
        else:
            lo = mid + 1
    return lo

def n_chebyshev_for_target(delta, T=T, p=p, sigma2=sigma2):
    mu = 2*p - 1
    A = mu*mu
    B = -(2*mu*T + sigma2/delta)
    C = T*T
    disc = B*B - 4*A*C
    n_root = (-B + math.sqrt(disc)) / (2*A)    
    return max(math.ceil(n_root), math.floor(T/mu) + 1)

for d in [0.05, 0.01, 0.001]:
    n_h = n_hoeffding_for_target(d)
    n_c = n_chebyshev_for_target(d)
    print(f"δ={d:>7}:  Hoeffding n={n_h:5d}  (bound={hoeffding_tail(n_h):.3g})   "
          f"Chebyshev n={n_c:5d}  (bound={chebyshev_tail(n_c):.3g})")

δ=   0.05:  Hoeffding n=  657  (bound=0.0497)   Chebyshev n= 1286  (bound=0.05)
δ=   0.01:  Hoeffding n=  701  (bound=0.00995)   Chebyshev n= 3325  (bound=0.01)
δ=  0.001:  Hoeffding n=  756  (bound=0.000973)   Chebyshev n=24990  (bound=0.001)
